## Generating Synthetic Data

In [1]:
import pandas as pd
from os.path import join

In [2]:
DATA_FOLDER = '/Users/rlopez/PTG/tim-reasoning/data/object_states'
ANNOTATIONS_CSV = join(DATA_FOLDER, 'formatted_annotations.csv')

OBJECTS = ['tortilla', 'plate']
annotations = pd.read_csv(ANNOTATIONS_CSV)

annotations.head(30)

,recipe,video_name,start_time,end_time,step,tortilla,plate
0,pinwheels,pinwheels_2023.03.30-16.38.48,0,7,place tortilla,in-package,empty
1,pinwheels,pinwheels_2023.03.30-16.38.48,8,22,place tortilla,plain,empty
2,pinwheels,pinwheels_2023.03.30-16.38.48,23,32,spread PB,plain,empty
3,pinwheels,pinwheels_2023.03.30-16.38.48,33,49,spread PB,peanut-butter[initial],empty
4,pinwheels,pinwheels_2023.03.30-16.38.48,50,83,spread PB,peanut-butter[full],empty
5,pinwheels,pinwheels_2023.03.30-16.38.48,84,97,clean knife,peanut-butter[full],empty
6,pinwheels,pinwheels_2023.03.30-16.38.48,98,109,spread J,peanut-butter[full],empty
7,pinwheels,pinwheels_2023.03.30-16.38.48,110,118,spread J,pb+jelly[partial],empty
8,pinwheels,pinwheels_2023.03.30-16.38.48,119,138,spread J,pb+jelly[full],empty
9,pinwheels,pinwheels_2023.03.30-16.38.48,139,149,clean knife,pb+jelly[full],empty


In [3]:
import copy
import json
from datetime import datetime

CURRENT_TIME = int(datetime.now().timestamp())

def select_tracked_objects(row, objects):
    tracked_objects = {}
    
    for obj in objects:
        if row[obj] != '':
            tracked_objects[obj] = row[obj]
    
    return tracked_objects

def get_unique_states(annotations, objects):
    unique_states = {}
    
    for obj in objects:
        states = annotations[obj].unique()
        unique_states[obj] = list(states)
    
    return unique_states

def read_annotated_video(annotations, recipe_id, video_id):
    step_id = 0
    current_step = None
    unique_states = get_unique_states(annotations[annotations['recipe'] == recipe_id], OBJECTS)    
    video_annotations = annotations[annotations['video_name'] == video_id]
    annotated_video = {'task_id': recipe_id, 'session_id': video_id, 'history': {}, 'unique_states': unique_states}
    
    for _, row in video_annotations.iterrows():
        if current_step != row['step']:
            current_step = row['step']
            step_id += 1
        
        tracked_objects = select_tracked_objects(row, OBJECTS)
        
        if step_id not in annotated_video['history']:
            annotated_video['history'][step_id] = []
        annotated_video['history'][step_id].append({'start_time': row['start_time'], 'end_time': row['end_time'], 'objects': tracked_objects})
    
    return annotated_video

def make_perception_outputs(annotated_video):
    PERCEPTION_OUTPUT_TEMPLATE = {
        "pos": [-0.2149151724097291, -0.4343880843796524, -0.6208099189217009],
        "xyxyn": [0.1, 0.1, 0.2, 0.2],
        "label": "",
        "id": "",
        "status": "tracked",
        "last_seen":"",
        "state": {},
        "hand_object_interaction": 0.27
    }
    
    perception_outputs = []
    
    for step_id, step_annotations in annotated_video['history'].items():
        for step_annotation in step_annotations:
            step_outputs = make_step_outputs(step_annotation, annotated_video['unique_states'], PERCEPTION_OUTPUT_TEMPLATE)
            perception_outputs += step_outputs
    
    return perception_outputs

def make_step_outputs(step_annotation, unique_states, output_template):
    objects = step_annotation['objects']
    start_time = step_annotation['start_time']
    end_time = step_annotation['end_time']
    step_outputs = []
    
    for time_secs in range(start_time, end_time+1):
        time_stamp = CURRENT_TIME + time_secs
        for object_name, object_state in objects.items():
            object_output = copy.deepcopy(output_template)
            object_output['label'] = object_name
            object_output['last_seen'] = time_stamp
            state_probas = {s: 0.0 for s in unique_states[object_name]}
            state_probas[object_state] = 1.0
            object_output['state'] = state_probas
            step_outputs.append(object_output)
            
    return step_outputs

def save_outputs(outputs, file_name):
    with open(join(DATA_FOLDER, f'{file_name}.json'), 'w') as fout:
        json.dump(outputs, fout, indent=2)


In [4]:
annotated_video = read_annotated_video(annotations, 'pinwheels', 'pinwheels_2023.03.30-16.38.48')

In [5]:
perception_outputs = make_perception_outputs(annotated_video)
save_outputs(perception_outputs, 'perception')

In [6]:
def make_errors():
    pass

def mix_tasks():
    pass

def evaluate_reasoning():
    pass